# <font color='#3E1485'>**Toxic Comment Classification Challenge**</font>


## <font color='#9271C3'>背景</font>

多くのオンラインウェブサイト、アプリケーション、サービスでは、ディスカッション、コメント、その他の目的でユーザーがテキストを入力することができます。ほとんどの場合、私たちはユーザーが適切に行動し、テキスト機能を乱用しないことを期待しています。しかし、時には意図的であろうとなかろうと、ユーザーが会話に貢献しないものを入力し、他の人の体験を悪化させる結果になりかねません。

この種のチャット・サービスを運営する多くの事業体にとって、会話を監視するための自動化ツールは需要が高い。有害な言葉を識別するためのそのようなツールは、すべてのユーザーにとってより良い、より一貫性のあるエクスペリエンスにつながるだろう。

## <font color='#9271C3'>タスク</font>

Wikipediaから収集されたコメントが提供される。様々なタイプの有害コメントを区別できるモデルを訓練せよ。

## <font color='#9271C3'>データ</font>

* `train.csv` - コメントとそのラベルを含むファイル。
* `test.csv` - テストデータを含むファイル。
* `sample_submission.csv` - サンプル提出ファイルを含むファイルであり、提出ファイルで期待されるカラムを示すために使用される。
* `test_labels.csv` - コンペティション終了後に公開されたテストデータのラベル。

## <font color='#9271C3'>お役立ちリンク集</font>

[コンペHP](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge)

[データ解説](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data)

[1位解答](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/discussion/52557)

[2位解答](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/discussion/52612)

[3位解答](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/discussion/52762)

# <font color='#3E1485'>**セットアップ**</font>

以下のセルは必要なデータをダウンロードし、ノートブックで使用する環境を設定します。

## <font color='#9271C3'>GPUランタイムを有効にする</font>

このノートブックはGPUを搭載したシステムで実行するように設計されています。ランタイムのタイプをGPUを含むものに切り替えていることを確認してください（例：A100、V100）。

GPUランタイムを使用しているかどうかを確認するには、次のセルを実行してください。フォーマットされたテキストが複数行出力されるはずです。

In [ ]:
!nvidia-smi

## <font color='#9271C3'>データセットをダウンロードする</font>

Kaggleはコンペティションと簡単にやり取りできるAPIを提供しています。私たちはこのAPIを使って自動的にデータをダウンロードし、予測をアップロードします。

このAPIを使用する最初のステップは、自分のユーザーとして認証することです。APIトークンはユーザー名とKaggleが生成したキーを含むファイルです。トークンはアカウントページからダウンロードすることができ、通常 `kaggle.json` と呼ばれます。APIトークンは、あなたのユーザーとしてAPIにアクセスするために必要なものなので、個人のGoogle Driveフォルダに安全に保管してください。

このノートブックはGoogle Driveフォルダ内の`kaggle.json`というKaggle APIトークンを検索します。トークンをGoogle Driveに置いたことを確認し、プロンプトが表示されたらこのノートブックがトークンにアクセスすることを許可してください。

認証後、参加したすべてのコンペティションにアクセスできます。データのダウンロードにエラーが発生した場合は、Kaggle API トークンが有効であること、コンペティションのルールに同意していることを確認してください。

In [ ]:
from google.colab import drive
import os
import json

drive.mount("/content/drive", force_remount=True)
fin = open("/content/drive/MyDrive/kaggle.json", "r")
json_data = json.load(fin)
fin.close()
os.environ["KAGGLE_USERNAME"] = json_data["username"]
os.environ["KAGGLE_KEY"] = json_data["key"]

In [ ]:
%%bash
kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
if [ $? -ne 0 ]; then
    echo "データのダウンロードに問題があったようです。"
    echo "競技規則に同意し、APIキーが有効であることを確認してください。"
else
    mkdir -p /content/kaggle
    unzip -o /content/jigsaw-toxic-comment-classification-challenge.zip -d /content/kaggle
    unzip -o "/content/kaggle/*.zip" -d /content/kaggle
    rm /content/kaggle/*.zip
fi
wget -q -P /tmp https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKjp-hinted.zip
unzip -o /tmp/NotoSansCJKjp-hinted.zip -d /usr/share/fonts/NotoSansCJKjp

## <font color='#9271C3'>環境</font>

デフォルトではインストールされていないライブラリを使用するので、ここでインストールします。

In [ ]:
!pip install transformers pytorch-lightning

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
import torch
import pytorch_lightning as pl
from tqdm.auto import tqdm
import torchmetrics
from glob import glob

font_path = '/usr/share/fonts/NotoSansCJKjp/NotoSansMonoCJKjp-Regular.otf'
matplotlib.font_manager.fontManager.addfont(font_path)
prop = matplotlib.font_manager.FontProperties(fname=font_path)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = prop.get_name()
os.chdir('/content/kaggle')

# <font color='#3E1485'>**視覚化**</font>

この大会のデータはコメントとそのラベルで構成されている。可視化できるものはあまりありませんが、ラベルの分布といくつかの有害なコメントを見ることができます。

## <font color='#9271C3'>train.csv</font>

In [ ]:
data = pd.read_csv('/content/kaggle/train.csv')
data

まず、ラベルの分布を見てみよう。

In [ ]:
data_bin = data.copy()
data_bin['non-toxic'] = 1 - data_bin[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
data_bin['toxic'] = data_bin[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
data_bin = data_bin[['id', 'toxic', 'non-toxic']].melt(id_vars='id')
sns.barplot(data_bin, x='variable', y='value', errorbar=None)
plt.xlabel(None)
plt.ylabel(None)
plt.xticks([0, 1], ['Toxic', 'Non-toxic']);

この図から、有害と分類されるコメントは全体の10％程度であることがわかる。

In [ ]:
data_melt = data.copy()
data_melt = data_melt.drop(columns=['comment_text']).melt(id_vars='id')
sns.barplot(data_melt, x='variable', y='value', estimator='sum', errorbar=None)
plt.xlabel(None)
plt.ylabel(None);

有害なコメントの種類は同じ頻度ではない。

In [ ]:
data_sum = data.copy()
data_sum['num_labels'] = data_sum[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)
data_sum = data_sum[data_sum['num_labels']>0]
sns.barplot(data_sum, x='num_labels', y='num_labels', estimator='count', errorbar=None)
plt.ylabel(None);

毒性クラスが通常より多く表示される理由のひとつは、毒性クラスが最も一般的な分類であり、コメントによっては複数の毒性タイプが含まれているからかもしれない。

有害なコメントを見てみよう。

In [ ]:
obscene = data.loc[67329]
obscene['comment_text']

このコメントはわいせつに分類されます。わいせつなコメントには通常、閲覧者によっては不適切と思われる言葉が含まれています。

In [ ]:
insult = data.loc[106700]
insult['comment_text']

このコメントは侮辱に分類される。侮辱は通常、1人または複数の人に向けられ、軽蔑や軽蔑を伝えます。

# <font color='#3E1485'>**モデリング**</font>

このノートブックでは、2018年に導入された人気の言語モデルであるBERTを使用します。また、HuggingFaceの`transformers` ライブラリも利用します。これは、事前に学習されたBERTモデルと、テキストデータを前処理するための様々なユーティリティを提供します。最後に、PyTorchとPyTorch Lightningを使用します。PyTorchとPyTorch Lightningは、モデルの作成と学習を可能にするディープラーニングによく使用されるライブラリです。

## <font color='#9271C3'>前処理</font>

BERT のようなモデルを使用するには、まず入力をトークン化する必要があります。これは単にテキストを BERT が理解できる数値に変換することを意味します。`transformers`は自動的に正しいトークナイザーを読み込む`AutoTokenizer`クラスを提供する。

次に、学習中に使用できる構造化された方法でデータを整理する必要があります。そのために、PyTorchの `Dataset` を使います。この `Dataset` はモデルとデータの間のインターフェイスとして機能します。

このコンペティションのデータセットは小さいので、すべてのトレーニングデータセットを前処理してメモリに保存することができます。そうすることで、データをその場でトークン化するのではなく、事前にトークン化されたデータをメモリからロードするため、トレーニングが大幅にスピードアップする。

トレーニングの後半では、データサンプルをバッチ処理する。簡単のため、トークナイザーが返す最初の128個のトークンのみを使用します。それ以上のトークンを持つテキストは切り捨てられ、それ以下のトークンを持つテキストはパディングされます。パラメータ `max_length` を変更すると、トークン化された入力により多くの情報が含まれるようになりますが、その分メモリのコストが増加します。

前処理には数分かかるかもしれない。

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, comments, labels=None):
        self.tokenized = []
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.float32)
        else:
            self.labels = None
        print('データのトークン化')
        for comment in tqdm(comments):
            self.tokenized.append({
                key: value[0] for key, value in
                    tokenizer(
                        comment,
                        return_tensors='pt',
                        padding='max_length',
                        truncation=True,
                        max_length=128
                    ).items()
            })

    def __len__(self):
        return len(self.tokenized)

    def __getitem__(self, idx):
        if self.labels is None:
            return self.tokenized[idx]
        else:
            return self.tokenized[idx], self.labels[idx]

ds = CustomDataset(tokenizer, data['comment_text'], data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values)

## <font color='#9271C3'>トレーニング</font>

今回使用するBERTモデルは、別のタスクのためにすでに訓練されています。単にモデルを初期化し、データセットのクラスの値を出力するように変更します。その後、モデルを微調整し、タスクのためにモデルを再トレーニングします。

標準的なPyTorchでは、モデルやタスクに関係なく同じテンプレートコードを書きます。これは非常に面倒であり、また細かい部分を忘れやすいためエラーが起こりやすい。PyTorch Lightningを使えば、学習コードの最も関連性の高い部分を1つのクラスにまとめ、細かい部分はライブラリに任せることができます。さらに、PyTorch Lightningはロギング、チェックポイント、その他のタスクのための多くの便利なユーティリティを提供します。

以下のセルでは、以下のメソッドを含む PyTorch Lightning `LightningModule` を定義します：
- `__init__` - 学習中に使用する変数を初期化します。これにはモデルとトレーニングの進捗を監視するためのメトリクスが含まれます。
- `forward` - 与えられた入力をどのようにモデルに渡すかを定義する。
- `(training|validation|predict)_step` - 各ステップで実行するアクションを定義します。最も単純なケースでは `forward` を呼び出して損失を計算するが、追加の後処理やロギングを含めることもできる。
- `configure_optimizers` - トレーニング中に使用するオプティマイザを返す。

学習を高速化し、オーバーフィッティングを回避するために、既に学習済みのモデルのパラメータは学習しない。学習されるのは、分類タスク用に作成されたパラメータのみである。

In [ ]:
class CustomModel(pl.LightningModule):
    def __init__(self, freeze_pretrained=True, lr=1e-5):
        super().__init__()
        self.lr = lr
        self.model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)
        if freeze_pretrained:
            for name, parameter in self.model.named_parameters():
                if 'classifier' not in name:
                    parameter.requires_grad = False
        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        self.train_AUROC = torchmetrics.classification.MultilabelAUROC(num_labels=6)
        self.val_AUROC = torchmetrics.classification.MultilabelAUROC(num_labels=6)

    def forward(self, tokenized_input):
        return self.model(**tokenized_input)

    def training_step(self, batch, batch_idx):
        tokenized_input, labels = batch
        preds = self.forward(tokenized_input).logits
        loss = self.loss_fn(preds, labels)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.train_AUROC(preds.sigmoid(), labels.long())
        self.log('train_AUROC', self.train_AUROC, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        tokenized_input, labels = batch
        preds = self.forward(tokenized_input).logits
        loss = self.loss_fn(preds, labels)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.val_AUROC(preds.sigmoid(), labels.long())
        self.log('val_AUROC', self.val_AUROC, on_step=False, on_epoch=True)

    def predict_step(self, batch, batch_idx):
        tokenized_input = batch
        preds = self.forward(tokenized_input).logits
        preds = preds.sigmoid()
        return preds.detach().cpu()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

モデルが学習しているかどうかを確認するには、トレーニング中のパフォーマンスをモニターする必要がある。具体的には、時間の経過とともに損失が減少していること、追跡しているメトリクスに改善が見られることを確認したい。

ここでは、TensorBoardを使用してトレーニングを監視します。TensorBoardは、時間の経過とともにモデルのパフォーマンスを可視化する便利なツールです。

TensorBoardサーバーを起動するには、以下のセルを実行します。数秒後、インターフェースが表示されます。トレーニングやロギング値を開始していないため、最初は何も出力されません。トレーニングが開始されると、セルの出力は定期的に更新されます。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

トレーニングの準備はほぼ整いました。次のセルでは、まずデータを学習セットと検証セットに分割し、データをバッチ処理するPyTorchの `DataLoaders` を準備します。次に、モデルとロガーを初期化し、モデルの最適な反復を保存するコールバックを作成します。最後に、学習を開始します。

In [ ]:
train_ds, val_ds = torch.utils.data.random_split(ds, [0.8, 0.2])
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=False)

model = CustomModel()
tb_logger = pl.loggers.TensorBoardLogger('lightning_logs', name='', version='custom_model')
if os.path.exists(f'lightning_logs/{tb_logger.version}/checkpoints'):
    for checkpoint in os.listdir(f'lightning_logs/{tb_logger.version}/checkpoints/'):
        os.remove(f'lightning_logs/{tb_logger.version}/checkpoints/{checkpoint}')
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_AUROC', mode='max')
profiler = pl.profilers.AdvancedProfiler(dirpath='.', filename='profile')
trainer = pl.Trainer(
    max_epochs=3, accelerator='gpu', precision='16-mixed',
    logger=tb_logger,
    callbacks=[checkpoint_callback],
    enable_progress_bar=True
)

trainer.fit(model, train_dl, val_dl)

## <font color='#9271C3'>予測</font>

モデルの学習が終了したら、テストデータの予測を生成することができます。以前のようにデータをトークン化する必要があります。さらに、さまざまなトレーニング・パラメータによっては、モデルが最適な状態にないこともあります。しかし、チェックポイント・コールバックで最適なバージョンを保存したので、簡単にモデルを最適な状態にリロードすることができます。

In [ ]:
test_data = pd.read_csv('test.csv')
test_ds = CustomDataset(tokenizer, test_data['comment_text'])
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=False)
ckpt_path = glob(f'lightning_logs/{tb_logger.version}/checkpoints/*')[0]
model = CustomModel.load_from_checkpoint(ckpt_path)
preds = trainer.predict(model, test_dl)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['id'] = test_data['id']
submission[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] = torch.vstack(preds).numpy()
submission.to_csv('submission.csv', index=False)
submission

## <font color='#9271C3'>Kaggleへのアップロード</font>

予測はKaggleのAPIを通して直接提出されるため、ファイルを手動でダウンロードし、Kaggleのウェブサイトに再アップロードする必要はありません。

In [ ]:
!kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f submission.csv -m "これはアップロードに添付されたメッセージである。"

[提出ページ](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/submissions)でスコアを確認する。

# <font color='#3E1485'>**提案**</font>

**他のモデル** - BERT以外にも多くの言語モデルがあります。新しいモデルの方が性能は良いですが、パラメータが多いのでオーバーフィッティングになりやすいかもしれません。

**学習率** - 我々の学習率は非常に小さい。これはオーバーフィッティングを避けるのに役立ち、スコアが悪くなる可能性がありますが、訓練時間が長くなります。学習結果を観察し、学習率を上げてみてください。

**エポック** - 我々のモデルは3エポックしか学習しません。TensorBoardでトレーニングの進捗を確認すると、検証損失とメトリクスが着実に向上しているように見えますが、これはもっとトレーニングすべき指標です。エポック数を増やしてみましょう。

**すべてのパラメータを訓練する** - 現在、分類器の重みのみを更新しており、事前訓練された重みは更新していません。すべての重みを更新することで、より高いスコアが得られますが、オーバーフィットしやすくなります。さらに、すべてのパラメータをトレーニングすると、トレーニング時間とメモリ要件が増加します。

**トレーニングの自動停止** - 多くのエポック数を学習すると、モデルの改善はどこかで止まってしまいます。PyTorch Lightningには、学習を早期に終了させる早期停止コールバックがあります。[ドキュメンテーション](https://lightning.ai/docs/pytorch/stable/common/early_stopping.html)を確認して、学習プロセスに追加できるか確認してください。